In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""Add floor numbers to HCMC dataset"""

__author__ = "Anna Buch, Heidelberg University"
__email__ = "anna.buch@uni-heidelberg.de"

# Floor numbers and geolocations

**Aim**: \
Try to get the number of floors per shophouse and the rather exact geolocations of the shophouses, in which the surveyed microbusinesses are located in.
Due that the GPS coordinates in the HCMC dataset are unprecise, the matching has to be done based on the building addresses.

First examine if the shophouse addresses from the HCMC dataet matches with the addresses from a second dataset comprising housing information and geolocation.
Update the HCMC dataset for the addresses which occur in both datasets. By doing this the floor number and geolocation can be obtained for at least some records.
For the remaining records (shophouses for which no floor number exist) assume a two-storey building, based on the findings from Moon et al. 2009.

In [ ]:
import sys, os

import numpy as np
import pandas as pd
import geopandas as gpd
import re

import matplotlib.pyplot as plt
import seaborn as sns


UTILS_PATH = os.path.join(os.path.abspath(''), '../', 'utils')
sys.path.append(UTILS_PATH)
import figures as f
import preprocessing as pp
import feature_selection as fs


import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)


In [ ]:
geoinformations = gpd.read_file("../input_survey_data/Buildings_HCMC/HCMC_buildings_survey_TUEB_addresses.shp")  # Dataset in vietnamese characters

## data cleaning and repair
geoinformations.Nr_Floors = geoinformations.Nr_Floors.replace({10:1, 20:2, 30:3, 40:4}).astype("Int64")  ## fix floor numbers. 10-->1, 20-->2
geoinformations["Street"] = geoinformations["Street"].str.lower().replace(",", "")

geoinformations.head(3)

In [ ]:
# load survey data
raw_data = pd.read_excel("../input_survey_data/all-attributes_shophouses.xlsx")  # Niveditas dataset

## vietnamese version of survey data
vietnamese_data = pd.read_excel("../input_survey_data/Data DECIDER shophouse.xlsx")  # Dataset in vietnamese characters
vietnamese_data.head(3)

## data cleaning and repair
raw_data["Q0.3"] = raw_data["Q0.3"].str.lower().replace(",", "")
vietnamese_data["Q0.3"] = vietnamese_data["Q0.3"].str.lower().replace(",", "")
raw_data
## Fix erroneous coordinate pair by removing second decimal point
raw_data.GPS = raw_data.GPS.astype(str).replace({"10.722.546,106.62888":"10.722546,106.62888",
                                     "10797626106701100":"10.797626,106.701100",  # idx 24
                                     "10722187106.63":"10.722187,106.63"})  # idx 152

### update dataset with vietnamese addresses

In [ ]:
print(raw_data.shape)
#t = FuzzyMerge(left=raw_data, right=vietnamese_data, left_on="GPS", right_on="GPS").main()
raw_data["id"] = raw_data.index
raw_data.insert(0, "id", raw_data.pop("id"))  # use key to identify doublicates after merged with vietnamese df

raw_data["Q0.3"] =  raw_data["Q0.3"].str.strip()
vietnamese_data["Q0.3"] = vietnamese_data["Q0.3"].str.strip()
vietnamese_data["Housenumber_street"] = vietnamese_data["Q0.3"]
print(len(vietnamese_data["Housenumber_street"].unique()))
raw_data["housenumber_street"] = raw_data["Q0.3"]
print(len(raw_data["housenumber_street"].unique()))

raw_data_vietnamese = pp.FuzzyMerge(left=raw_data, right=vietnamese_data[["Housenumber_street","Q0.2.District", "Q0.2.Ward", "Q0.3"]], left_on="Q0.3", right_on="Q0.3").main()
#raw_data_vietnamese = pp.FuzzyMerge(left=raw_data, right=vietnamese_data[["Housenumber_street","Q0.2.District", "Q0.2.Ward", "Q0.3", "Q0.5"]], left_on="Q0.3", right_on="Q0.3").main()
print(raw_data_vietnamese.shape)
print(len(raw_data_vietnamese["housenumber_street"].unique()))
print(len(raw_data_vietnamese["Housenumber_street"].unique()))  # all ientmaese adresses matched with its corresponding address from Niveditas dataset

## TODO robustify by replacing all columns ending with "_x" by columns ending with "_y"
raw_data_vietnamese["housenumber_street"] = raw_data_vietnamese["Housenumber_street"]
raw_data_vietnamese["Q0.2.District_x"] = raw_data_vietnamese["Q0.2.District_y"]
raw_data_vietnamese["Q0.2.Ward_x"] = raw_data_vietnamese["Q0.2.Ward_y"]
#raw_data_vietnamese["Q0.5_x"] = raw_data_vietnamese["Q0.5_y"]
raw_data_vietnamese.drop(["Housenumber_street", "Q0.2.District_y","Q0.2.Ward_y"], axis=1, inplace=True)
#raw_data_vietnamese.drop(["Housenumber_street", "Q0.2.District_y","Q0.2.Ward_y", "Q0.5_y"], axis=1, inplace=True)
raw_data_vietnamese.columns = raw_data_vietnamese.columns.str.rstrip('_x')
print(raw_data_vietnamese.shape)
raw_data_vietnamese

## 10 records are dublicates in columns which are from raw_data

In [ ]:
raw_data_vietnamese[raw_data_vietnamese.id.duplicated(keep=False)]  # show all dublicates
#(raw_data_vietnamese.loc[[147]].values == raw_data_vietnamese.loc[148].values).all()


In [ ]:
## remove dublicated records
raw_data_vietnamese = raw_data_vietnamese[~raw_data_vietnamese.duplicated(keep="last")]
print(raw_data_vietnamese.shape)
raw_data_vietnamese.drop("id",axis=1, inplace=True)

In [ ]:
# get coords in readable format for gpd
raw_data_vietnamese = gpd.GeoDataFrame(raw_data_vietnamese,  
            geometry=gpd.points_from_xy( 
                    raw_data_vietnamese["GPS"].str.split(",").str[1], # lon
                    raw_data_vietnamese["GPS"].str.split(",").str[0],  # lat
            )
        )
print(raw_data_vietnamese.shape)

## save shp locations to disk, 
# extract elevation based on shop locations in datapoints_vars_bui.shp via QGIS due to loading size and process with gdal
print(raw_data_vietnamese.crs)
raw_data_vietnamese = raw_data_vietnamese.set_crs(4326) 

In [ ]:
raw_data_vietnamese[~raw_data_vietnamese.is_empty].geometry.info()
# raw_data_vietnamese.geometry.info(9)

### Overview

In [ ]:
## visual check of SMEs locations
import folium

glimpse = raw_data_vietnamese[~ raw_data_vietnamese.is_empty]   # drop emtpy geoms
glimpse_geolocations = geoinformations[~ geoinformations.is_empty]   # drop emtpy geoms
m = glimpse.geometry.explore(name="survey ds", color="red", k=6)  
m = glimpse_geolocations.explode(ignore_index=True).explore(
    m=m, 
    name="geolocations",
    column="Nr_Floors", 
    popup=True, 
    #tooltip="Nr_Floors", 
    #cmap="winter"
    cmap="Set1"
)    # BT_Moon BT_TUEB_2

folium.LayerControl().add_to(m)
m

In [ ]:
### Distribution of floor numbers
 
t = geoinformations.groupby(["BT_Moon", "Nr_Floors"]).size().unstack(0)
t.plot.bar(stacked=True)

#plt.hist(stacked=True)

#### Join adresses
Add geolocations and building information based on common street and house numbers to the updated survey dataset

In [ ]:
raw_data_vietnamese["housenumber_street"] = raw_data_vietnamese["housenumber_street"].str.strip()
#raw_data_vietnamese["street_housenumber"].isna().sum()
geoinformations['Housenumber_street'] = geoinformations['HouseNumbe'] + " " + geoinformations['Street']

## drop unknown adresses 
print(f"Removing {geoinformations.Housenumber_street.isna().sum()} records with missing address")
geoinformations = geoinformations.loc[~geoinformations.Housenumber_street.isna(),:]
#geoinformations.street_housenumber

In [ ]:
## make street names more similar, due that Tuebingen group only used first two words of each street name, but in Niveidtas dtataset the entire street names are used
raw_data_vietnamese.housenumber_street = raw_data_vietnamese.housenumber_street.astype(str)
for idx, street in enumerate(raw_data_vietnamese.housenumber_street):
    if street != "nan":
        print(street)
        raw_data_vietnamese.housenumber_street[idx] = ' '.join([x if index != 3 else "" for index, x in enumerate(street.split())]).strip()

#raw_data_vietnamese_geolocations.housenumber_street.unique()

*Note:* Housenumber how to read: e.g. “25/1/10” means “house No.10 in niche 1 of alley 25”

In [ ]:
raw_data_vietnamese["housenumber_street"].unique()  # --> 62 records with adrresses

In [ ]:
geoinformations["id"] = geoinformations.index  # get unique key to access number of matches
raw_data_vietnamese["id_x"] = raw_data_vietnamese.index
raw_data_vietnamese.insert(0, "id_x", raw_data_vietnamese.pop("id_x"))

geoinformations['updated_geometry'] = geoinformations["geometry"]

## NOTE test to repair
raw_data_vietnamese_geolocations = pd.merge(
    left=raw_data_vietnamese, #["housenumber_street"], 
    right=geoinformations[['BT_Moon', 'Nr_Floors', 'BT_TUEB_2','Housenumber_street', 'id', 'updated_geometry']], 
    left_on="housenumber_street", right_on="Housenumber_street",
    how="left")


print(raw_data_vietnamese.shape)
print(geoinformations.shape)
print(raw_data_vietnamese_geolocations.shape)
print(len(geoinformations.id.unique()))
print(len(raw_data_vietnamese_geolocations.id.unique()))  # records with improved geolocations

## count records which didnt match with any of the geolocations , = missing ids after joined adresses
print(raw_data_vietnamese_geolocations.id.isna().sum())  # cutoff: 0.9 :177 , 0.8: 133 (477 records), 0.7:130 (531 records)

raw_data_vietnamese_geolocations.info()


In [ ]:
len(geoinformations.Nr_Floors) - geoinformations.Nr_Floors.isna().sum()

In [ ]:
geoinformations.Nr_Floors.isna().sum()

## update geometries 
Replace GPS location of records in survey dataset where a imporved geolocation from Tuebingen exists


In [ ]:
raw_data_vietnamese_geolocations.info()

In [ ]:
raw_data_vietnamese_geolocations.updated_geometry = raw_data_vietnamese_geolocations.updated_geometry.representative_point()


In [ ]:
raw_data_vietnamese_geolocations

In [ ]:

raw_data_vietnamese_geolocations['geometry'] = np.where(
    ~raw_data_vietnamese_geolocations['updated_geometry'].isnull(), 
    raw_data_vietnamese_geolocations['updated_geometry'],
    raw_data_vietnamese_geolocations['geometry']
)
# raw_data_vietnamese_geolocations['geometry']
raw_data_vietnamese_geolocations["P4Q4.2.1"] = raw_data_vietnamese_geolocations["P4Q4.2.1"].astype(str)
raw_data_vietnamese_geolocations.drop(["GPS", "Q0.10", "P4Q4.2.2","updated_geometry"], axis=1, inplace=True)

In [ ]:
## prepare for saving as shp
datetime_col = raw_data_vietnamese_geolocations.select_dtypes(include=['datetime64']).columns.to_list()
raw_data_vietnamese_geolocations[datetime_col] = raw_data_vietnamese_geolocations[datetime_col].astype(str)
raw_data_vietnamese_geolocations[["Q0.6","P1Q2.1.1", "P1Q2.2.1" ,"P1Q2.1.2","P1Q2.2.2", "P4Q4.2.1"]] = raw_data_vietnamese_geolocations[["Q0.6","P1Q2.1.1", "P1Q2.2.1" ,"P1Q2.1.2","P1Q2.2.2", "P4Q4.2.1"]].astype(str)

print(raw_data_vietnamese_geolocations.crs)
raw_data_vietnamese_geolocations = raw_data_vietnamese_geolocations.set_crs(4326) 

## save shp locations to disk
raw_data_vietnamese_geolocations.loc[:, "P1Q2.2.2":"geometry"].to_file('../input_survey_data/DEM_LiDAR/datapoints_vars_bui_tueb.shp')  

In [ ]:
## all dublicate columns which need to be corrected

raw_v_g_dubl = pd.concat(g for _, g in raw_data_vietnamese_geolocations.groupby("id_x") if len(g) > 1)
raw_v_g_dubl[["id_x" ,"Q0.14", "housenumber_street", "geometry",	"BT_Moon",	"Nr_Floors", "BT_TUEB_2", "Housenumber_street", "id"]]

## write to disk

In [ ]:
## save updated HCMC survey ds (with corrrect adresses, and partly improved geolocations)

## save shp locations to disk
raw_data_vietnamese_geolocations_sm = gpd.GeoDataFrame(
    pd.concat(  # TODO add columns numbers of candidate predictors + "BT_Moon",	"Nr_Floors",	"BT_TUEB_2"
        [raw_data_vietnamese_geolocations[["id_x"]], raw_data_vietnamese_geolocations.loc[:, "P1Q2.2.2":"geometry"]]
        , axis=1))
raw_data_vietnamese_geolocations_sm.to_file('../input_survey_data/DEM_LiDAR/datapoints_vars_bui_tueb_tst2.shp')  

In [ ]:
raw_data_vietnamese_geolocations[raw_data_vietnamese_geolocations.duplicated(subset=["id_x"],keep=False)].sort_values("id_x")

In [ ]:
raw_data_vietnamese_geolocations.shape

In [ ]:
## record number with missing imporved geolocation info
raw_data_vietnamese_geolocations.Housenumber_street.isna().sum()

In [ ]:
raw_data_vietnamese_geolocations.housenumber_street.isna().sum()

## Manual postprocessing
for a quicker processing all samples are written to excel, these samples are comprissed out of the survey shophouses and can occure one or more times depending how often a match with similar or identical addresses form the tuebing dataset ws found. 

*raw_data_vietnamese_geolocations_incl_dublicates.xlsx* is processed manually by keeping only records which have idnetical addresses or if addresses slightly differ a visual comparison of the adresses is done. If both addresses describe buildings next to each other than it is assumed that building tye and especially floor number is similar 

Manual created output: **raw_data_vietnamese_geolocations_no_dublicates.xlsx** which contains all records from Niveditas dataset, some of theses 252 shops (ie. records) contain improved geoinformation and building information such as floor number 


**TODO** 
make this in python e.g. by creating new column with same number for each group of dublicates and select out of each group the record with the fewest missing values

In [ ]:
## all dublicate columns which need to be corrected

raw_v_g_dubl = pd.concat(g for _, g in raw_data_vietnamese_geolocations.groupby("id_x") if len(g) > 1)
raw_v_g_dubl[["id_x" ,"Q0.14", "housenumber_street", "geometry",	"BT_Moon",	"Nr_Floors", "BT_TUEB_2", "Housenumber_street", "id"]]

In [ ]:
# raw_data
raw_data_vietnamese_geolocations